In [1]:
import os
import json

import numpy as np
import pandas


from keras.models import Sequential
from keras.layers import Dense, Conv2D, BatchNormalization, Activation, InputLayer, LeakyReLU, Reshape, Flatten
from keras.layers.pooling import MaxPooling1D, MaxPooling2D, AveragePooling1D, AveragePooling2D

from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras.optimizers import Adam

import librosa

Using TensorFlow backend.


In [2]:
def load_index(fold_id, key='train', n=None):
    
    with open(os.path.join(DESTDIR, 'experiment_{:02d}.json'.format(fold_id)), 'r') as fdesc:
        index = json.load(fdesc)[key]
    
    perm = np.random.permutation(len(index))
    if n is not None:
        perm = perm[:n]
        
    index = [index[_] for _ in perm]
    
    return index

In [3]:
def preprocess(X):
    return librosa.logamplitude(X**2, ref_power=np.max, top_db=80)


def load_data(index, n_harm=1):
    
    X, Y = [], []
    for item in index:
        with np.load(os.path.join(DESTDIR, 'features', '{}.npz'.format(item['filename']))) as data:
            if n_harm is None:
                n_harm = data['C'].shape[0]
            X.append(preprocess(data['C'][:n_harm]))
            Y.append(item['classID'])
    
    X = np.asarray(X).swapaxes(3, 1)
    Y = np.asarray(Y)
    
    return X, Y

In [4]:
DESTDIR = '/home/bmcfee/working/minst/'

In [5]:
FOLD_ID = 1

In [6]:
index = load_index(FOLD_ID)

In [7]:
test_index = load_index(FOLD_ID, key='test')

In [38]:
N_HARM = 4

In [39]:
X, Y = load_data(index, n_harm=N_HARM)

In [40]:
Xt, Yt = load_data(test_index, n_harm=N_HARM)

In [41]:
X.shape

(35480, 44, 252, 4)

In [42]:
model = Sequential()

model.add(InputLayer(input_shape=X.shape[1:], name='input'))
model.add(BatchNormalization())

model.add(Conv2D(16, 5, (4 - X.shape[3] + 1) * 5, bias=False))

model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))

model.add(MaxPooling2D(pool_size=(5, 5)))

model.add(Conv2D(32, 5, 5,  bias=False))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))

#model.add(MaxPooling2D(pool_size=(5, 5)))

model.add(Conv2D(64, 3, model.output_shape[2], bias=False))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))


model.add(AveragePooling2D(pool_size=(model.output_shape[1], 1)))
model.add(Flatten())

model.add(Dense(12, activation='softmax'))



In [43]:
model.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])

In [44]:
weight_fn = '/home/bmcfee/working/minst/models_{}.hdf5'.format(N_HARM)

In [45]:
history = model.fit(X, to_categorical(Y, nb_classes=12),
                    callbacks=[EarlyStopping('val_loss', patience=10),
                               ModelCheckpoint(weight_fn, save_best_only=True)],
                    validation_split=0.25, batch_size=32, shuffle=True,
                    nb_epoch=30)

Train on 26610 samples, validate on 8870 samples
Epoch 1/30
26610/26610 [==============================] - 34s - loss: 0.3086 - acc: 0.9130 - val_loss: 0.1231 - val_acc: 0.9625
Epoch 2/30
26610/26610 [==============================] - 33s - loss: 0.0802 - acc: 0.9779 - val_loss: 0.0668 - val_acc: 0.9781
Epoch 3/30
26610/26610 [==============================] - 33s - loss: 0.0473 - acc: 0.9860 - val_loss: 0.0569 - val_acc: 0.9829
Epoch 4/30
26610/26610 [==============================] - 33s - loss: 0.0384 - acc: 0.9888 - val_loss: 0.0370 - val_acc: 0.9909
Epoch 5/30
26610/26610 [==============================] - 33s - loss: 0.0272 - acc: 0.9918 - val_loss: 0.0493 - val_acc: 0.9835
Epoch 6/30
26610/26610 [==============================] - 33s - loss: 0.0263 - acc: 0.9923 - val_loss: 0.0292 - val_acc: 0.9921
Epoch 7/30
26610/26610 [==============================] - 33s - loss: 0.0229 - acc: 0.9926 - val_loss: 0.0405 - val_acc: 0.9873
Epoch 8/30
26610/26610 [==============================]

In [46]:
model.load_weights(weight_fn)

In [17]:
print(N_HARM)
model.evaluate(Xt, to_categorical(Yt))

1
3417/3417 [==============================] - 1s     


[2.574191954284514, 0.59262510936163237]

In [27]:
print(N_HARM)
model.evaluate(Xt, to_categorical(Yt))

2
3417/3417 [==============================] - 1s     


[2.0970507852508549, 0.59321041842598232]

In [37]:
print(N_HARM)
model.evaluate(Xt, to_categorical(Yt))

3
3417/3417 [==============================] - 1s     


[2.438846124868697, 0.58208955188993472]

In [47]:
print(N_HARM)
model.evaluate(Xt, to_categorical(Yt))

4
3417/3417 [==============================] - 1s     


[2.5461347863309243, 0.58940591139161247]